Лабораторная работа 7. Яровенко Максим, ИУ5Ц-102Б. Вариант 22

# **Задание 1**

В однопроцессорную  систему случайным образом поступают на выполнение $m$ разных типов программ, отличающихся известной трудоемкостью $Q_1, Q_2, \ldots, Q_m$. Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью, вычислить среднее время обслуживания программ, считая длительность обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ. Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет величина кванта на среднее время обслуживания программ.

Задание по варианту:

In [1]:
Variant<-22
set.seed(Variant)
m<-sample(c(6:20),1)
lambda<-runif(1,0.1, 2)
Q<-rexp(m,0.3)
q<-sample(c(1:4),1)
View(data.frame(m,q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
11,2,1.002004


 [1] 1.2029659 0.7677870 1.5944755 4.5403571 3.9454905 1.0944295 2.5005737
 [8] 5.9519855 0.7875043 3.9312316 5.4535731


Теоретическое решение

---



Одноканальная система с неограниченной очередью


$$\rho=\frac{\lambda}{\mu}$$


$$T_{\text{сист}}=\frac{1}{\mu(1-\rho)}=\frac{1}{\mu-\lambda}\tag{лек-3}$$

$$\mu_{ср} = \frac{1}{Q_{ср}} = \frac{n}{\sum_{i=1}^{q}{q_i}}$$

In [2]:
mu <- length(Q)/sum(Q); mu; '<'; lambda

[1] 0.3462345

[1] "<"

[1] 1.002004

$\mu_{ср}$ значительно меньше $\lambda$, поэтому очередь будет расти до бесконечности. **$\lambda$ нужно уменьшить**.

Пусть $\lambda = 0,3$

In [3]:
lambda = 0.3

print("Tsist теория")
print(1/(mu-lambda))

[1] "Tsist теория"
[1] 21.62886


Экспериментальное решение

---



Приближение к одноканальной СМО с беск очередью M/M/1/m

In [4]:
experimentApproximation <- function(lambda, mu, N) {
  event_task_recieve <- vector(mode='double', length=N)
  event_server_free <- vector(mode='double', length=N)
  event_wait_time <- vector(mode='double', length=N)
  event_queue_decrease_times <- vector(mode='double', length=N)
  time_free <- 0
  time_recieved <- 0
  time_required <- 0
  for (i in 1:N) {
    time_recieved <- time_recieved + rexp(1, lambda)
    event_task_recieve[i] <- time_recieved
    time_required <- rexp(1, mu)

    if (time_free < time_recieved) {
      time_free <- time_recieved + time_required
      event_wait_time[i] <- time_required
    } else {
      time_free <- time_free + time_required
      event_wait_time[i] <- time_free - time_recieved
      event_queue_decrease_times <- append(event_queue_decrease_times, time_free)
    }
    event_queue_decrease_times <- event_queue_decrease_times[event_queue_decrease_times > time_recieved]
    event_server_free[i] <- time_free
  }
  return(event_wait_time)
}

In [5]:
res1 <- experimentApproximation(lambda, mu, 1000000)
print("Tsist эксперимент приближение")
print(mean(res1))

[1] "Tsist эксперимент приближение"
[1] 22.52113


Понадобятся следующие методы для работы с очередью с приоритетом. Для SPT и RR приоритетнее тот, чьи проиоритет/время меньше.

In [6]:
create_queue<-function() {
  return(list(idx = c(), prior = c(), time = c()))
}

create_request<-function(idx, m, time) {
  return(list(idx = idx, prior = sample(1:m, 1), time = time))
}

push<-function(queue, idx, prior, time) {
  append(queue$idx, idx)
  append(queue$prior, prior)
  append(queue$time, time)
  return(queue)
}

remove<-function(queue, idx) {
  return(list(
    idx = queue$idx[-idx],
    prior = queue$prior[-idx],
    time = queue$time[-idx]
  ))
}

unshift<-function(queue) {
  return(remove(queue, 1))
}

pop_most_prior<-function(queue) {
  i <- which.min(queue$prior)
  return(list(
    most_prior = list(
      idx = queue$idx[i],
      prior = queue$prior[i],
      time = queue$time[i]
    ),
    queue = remove(queue, i)
  ))
}

In [7]:
experimentSTP <- function(lambda, Q, N) {
  Q <- sort(Q)
  event_wait_time <- vector(mode='double', length=N)
  time <- 0
  requests <- list(idx = c(1:N), prior = sample(c(1:length(Q)), N, replace=TRUE), time = rexp(N, lambda))
  for (i in 1:N) {
    time <- time + requests$time[i]
    requests$time[i] <- time
  }
  time <- 0
  time_free <- 0
  time_recieved <- 0
  time_required <- 0
  queue <- create_queue()
  i <- 1
  done_cnt <- 1
  while (T) {
    time_recieved <- requests$time[i]
    scenario <- which.min(c(time_free, time_recieved, Inf))
    if (scenario == 1) {
      time <- time_free
      if (length(queue$idx)) {
        pop_result <- pop_most_prior(queue)
        cur_request <- pop_result$most_prior
        queue <- pop_result$queue
        time_free <- time + Q[cur_request$prior]
        event_wait_time[done_cnt] <- time_free - cur_request$time
        done_cnt <- done_cnt + 1
      } else {
        if (is.na(time_recieved)) {
          # "все"
          break
        }
        time_free <- time_recieved + Q[requests$prior[i]]
        event_wait_time[done_cnt] <- time_free - time_recieved
        done_cnt <- done_cnt + 1
        i <- i + 1
      }
    } else if (scenario == 2) {
      time <- time_recieved
      if (length(queue$idx)) {
        queue <- push(queue, requests$idx[i], requests$prior[i], requests$time[i])
      } else {
        time_free <- time + Q[requests$prior[i]]
        event_wait_time[done_cnt] <- time_free - time_recieved
        done_cnt <- done_cnt + 1
      }
      i <- i + 1
    }
  }

  return(event_wait_time)
}

In [8]:
res2 <- experimentSTP(lambda, Q, 10000000)
res2 <- res2[res2 > 0]

print("Tsist эксперимент STP")
print(mean(res2))


[1] "Tsist эксперимент STP"
[1] 2.888605


RR

---



In [ ]:
experimentRR <- function(lambda, Q, q, N) {
  ti <- 0.01
  T <- N * ti
  time <- 0
  q_end_time <- 0
  cur_request<- NULL
  queue <- create_queue()
  event_wait_time <- c()
  requests <- list(idx = c(1:N), prior = sample(Q, N, replace=TRUE), time = rexp(N, lambda))
  for (i in 1:N) {
    time <- time + requests$time[i]
    requests$time[i] <- time
  }
  time <- 0
  i <- 1
  while (time < T - ti) {
    time = time + ti
    if (time > requests$time[i] && !is.na(requests$time[i])) {
      if (!is.null(cur_request)) {
        queue <- push(queue, requests$idx[i], requests$prior[i], requests$time[i])
      } else {
        cur_request <- list(idx=requests$idx[i], proc_left=requests$prior[i], time=requests$time[i], done_at=requests$time[i] + requests$prior[i])
        q_end_time <- time + q
      }
      i <- i + 1
    }
    if (time > cur_request$done_at && !is.null(cur_request)) {
      event_wait_time = append(event_wait_time, cur_request$done_at - cur_request$time)
      if (length(queue$idx)) {
        cur_request <- list(idx=queue$idx[1], proc_left=queue$prior[1], time=queue$time[1], done_at=cur_request$done_at + queue$prior[1])
        queue = remove(queue, 1)
        q_end_time <- time + q
      } else {
        cur_request = NULL
      }
    }
    if (time > q_end_time && !is.null(cur_request) && length(queue$idx)) {
      queue <- push(queue, cur_request$idx, cur_request$prior - q, cur_request$time)
      cur_request <- list(idx=queue$idx[1], proc_left=queue$prior[1], time=queue$time[1], done_at=queue$prior[1] + time)
      queue <- remove(queue, 1)
      q_end_time <- time + q
    }
  }
  return(event_wait_time)
}

In [ ]:
res3 <- experimentRR(lambda, Q, q, 1000000)
res3 <- res3[res3 > 0]
print("Tsist эксперимент RR")
print(paste('q =', q))
print(mean(res3))

res_rr <- mean(res3)

[1] "Tsist эксперимент RR"
[1] "q = 2"
[1] 2.910859


In [ ]:
for (i in (q+1):(q+5)) {
  res4 <- experimentRR(lambda, Q, i, 100000)
  res4 <- res4[res4 > 0]
  res_rr <- append(res_rr, mean(res4))
  print("Tsist эксперимент RR")
  print(paste('q =', i))
  print(res_rr[length(res_rr)])
}

res_rr_fr <- data.frame()
res_rr_fr <- rbind(res_rr_fr, res_rr)
colnames(res_rr_fr) <- q:(q+5)
View(res_rr_fr)

[1] "Tsist эксперимент RR"
[1] "q = 3"
[1] 3.054908
[1] "Tsist эксперимент RR"
[1] "q = 4"
[1] 3.053226
[1] "Tsist эксперимент RR"
[1] "q = 5"
[1] 2.656554
[1] "Tsist эксперимент RR"
[1] "q = 6"
[1] 2.836262
[1] "Tsist эксперимент RR"
[1] "q = 7"
[1] 3.07163


2,3,4,5,6,7
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.910859,3.054908,3.053226,2.656554,2.836262,3.07163


Итоговая таблица

---



In [ ]:
res <- data.frame(
  1/(mu-lambda),
  mean(res1),
  mean(res2),
  mean(res3)
)
colnames(res) <- c('Tsist теория','Tsist эксперимент приближение','Tsist эксперимент STP','Tsist эксперимент RR');
View(res)

Tsist теория,Tsist эксперимент приближение,Tsist эксперимент STP,Tsist эксперимент RR
<dbl>,<dbl>,<dbl>,<dbl>
21.62886,22.52113,2.888605,2.910859


# **Задание 2**

Оставаясь в условиях прошлой задачи в двухпроцессорную систему случайным образом поступают на выполнение $m$
 разных типов программ, отличающихся известной трудоемкостью $Q_1, Q_2, \ldots, Q_m$. Входящий поток простейший с интенсивностью $\lambda$
.

Представив данную систему как двухканальную СМО с неограниченной очередью, вычислить среднее время обслуживания программ, считая длительность обслуживания случайной величиной (теоретически и экспериментально) - не учитывая известные трудоемкости.

Реализовать алгоритм LPT, выбирая из очереди заявки в соответствии с их приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ. Сравнить полученные результаты.

In [ ]:
Variant<-22
set.seed(Variant)
m<-sample(c(6:20),1)
lambda<-runif(1,0.1, 2)
lambda = 0.3
Q<-rexp(m,0.3)

q<-sample(c(1:4),1)
View(data.frame(m,q, lambda))
View(Q)

#среднее время обслуживания
mu <- 1/(sum(Q)/length(Q))

m,q,lambda
<int>,<int>,<dbl>
11,2,0.3


[1] 1.2029659 0.7677870 1.5944755 4.5403571 3.9454905 1.0944295 2.5005737
 [8] 5.9519855 0.7875043 3.9312316 5.4535731

Теоретическое решение

---



Для нахождения времени пребывания заявки в системе воспользуемся следующими формулами из лекции №4
$$P_0=\left(\sum_{i=0}^n\frac{y^i}{i!}+\frac{y^{n+1}}{n!(n-y)}\right)^{-1}\tag{12}$$

$$L_{\text{оч}}=\frac{y^{n+1}}{n!\cdot n}\cdot P_0\cdot \frac{1}{(1-y/n)^2}=\frac{y^{n+1}P_0}{(n-1)!(n-y)^2}\tag{15}$$

$$L_{\text{сист}}=L_{\text{оч}}+y\tag{16}$$

$$W_{\text{сист}}=\frac{L_{\text{сист}}}{\lambda}\tag{18}$$

In [ ]:
# Вычисление вероятности P0
y <- lambda/mu
n <- 2
sum_term <- sum(y^seq(0, n, 1) / factorial(seq(0, n, 1)))
additional_term <- (y^(n + 1)) / (factorial(n) * (n - y))
P0 <- 1 / (sum_term + additional_term)

# Подсчет Lоч
numerator <- y^(n + 1) * P0
denominator <- factorial(n - 1) * (n - y)^2
Loch <- numerator / denominator

# Подсчет Lсист
Lsyst <- Loch +  y

# Среднее время обслуживания программ
Wsyst <- Lsyst / lambda

cat("Среднее время обслуживания программ:", Wsyst)


Среднее время обслуживания программ: 3.55556

Экспериментальное решение

In [ ]:
install.packages("simmer")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘Rcpp’




In [ ]:
library(simmer)

# Функция для генерации времени обслуживания программы
generate_program <- function() {
  program_type <- sample(1:m, 1)  # Выбор типа программы случайным образом
  service_time <- Q[program_type]  # Получение времени обслуживания программы из массива Q
  return(service_time)  # Возвращение времени обслуживания программы
}

# Определение очереди и траектории процесса
queue <- trajectory() %>%
  seize("processor", 1) %>%  # Захват процессора
  set_attribute("arrival_time", function() now(env)) %>%  # Установка атрибута времени прибытия
  set_attribute("service_time", function() generate_program()) %>%  # Установка атрибута времени обслуживания программы
  timeout_from_attribute("service_time") %>%  # Установка времени ожидания
  release("processor", amount = 1)  # Освобождение процессора

# Создание среды моделирования
env <- simmer() %>%
  add_resource("processor", capacity = 2) %>%  # Добавление ресурса процессора с емкостью 2
  add_generator("program", queue, function() rexp(1, lambda))  # Добавление генератора программ с экспоненциальным интервалом времени между появлениями

# Получение результатов моделирования
res_arrivals <- get_mon_arrivals(run(env, until = 10000))  # Запуск моделирования до времени 10000 и получение результатов

# Отображение результатов
Wsyst2 <- mean(res_arrivals$end_time - res_arrivals$start_time)  # Вычисление среднего времени ожидания
cat("Среднее время ожидания =", Wsyst2)  # Вывод среднего времени ожидания
View(res_arrivals)  # Отображение результатов в виде таблицы

Среднее время ожидания = 3.593829

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
program0,1.808451,3.011417,1.2029659,TRUE,1
program1,4.613201,10.066774,5.4535731,TRUE,1
program2,10.653495,16.605481,5.9519855,TRUE,1
program3,16.336777,22.288763,5.9519855,TRUE,1
program4,26.056049,30.001539,3.9454905,TRUE,1
program5,36.353572,38.854145,2.5005737,TRUE,1
program7,38.781457,39.621932,0.7677870,TRUE,1
program6,36.443512,41.897085,5.4535731,TRUE,1
program8,43.741952,44.944918,1.2029659,TRUE,1


Алгоритм LPT

---



In [ ]:
library(simmer)

# Функция для генерации времени обслуживания программы
generate_program <- function() {
  program_type <- sample(1:m, 1)  # Выбор типа программы случайным образом
  service_time <- Q[program_type]  # Получение времени обслуживания программы из массива Q
  return(service_time)  # Возвращение времени обслуживания программы
}

# Определение очереди и траектории процесса
queue <- trajectory() %>%
  seize("processor", 1) %>%  # Захват процессора
  set_attribute("arrival_time", function() now(env)) %>%  # Установка атрибута времени прибытия
  set_attribute("service_time", function() generate_program()) %>%  # Установка атрибута времени обслуживания программы
  timeout_from_attribute("service_time") %>%  # Установка времени ожидания
  release("processor", amount = 1)  # Освобождение процессора

# Создание среды моделирования
env <- simmer() %>%
  add_resource("processor", capacity = 2) %>%  # Добавление ресурса процессора с емкостью 2
  add_generator("program", queue, function() rexp(1, lambda))  # Добавление генератора программ с экспоненциальным интервалом времени между появлениями

# Получение результатов моделирования
res_arrivals <- get_mon_arrivals(run(env, until = 10000))  # Запуск моделирования до времени 10000 и получение результатов

# Отображение результатов
Wsyst3 <- mean(res_arrivals$end_time - res_arrivals$start_time)  # Вычисление среднего времени ожидания
cat("Среднее время ожидания =", Wsyst3)  # Вывод среднего времени ожидания
View(res_arrivals)  # Отображение результатов в виде таблицы

Среднее время ожидания = 3.368749

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
program0,0.09406572,1.188495,1.0944295,TRUE,1
program1,2.78852541,5.289099,2.5005737,TRUE,1
program2,3.57718807,8.117545,4.5403571,TRUE,1
program3,10.04121810,11.635694,1.5944755,TRUE,1
program4,12.94756880,13.735073,0.7875043,TRUE,1
program5,20.50832813,21.276115,0.7677870,TRUE,1
program6,22.43708880,26.382579,3.9454905,TRUE,1
program8,25.13084640,27.477009,1.0944295,TRUE,1
program7,23.92062412,28.460981,4.5403571,TRUE,1


Итоговая таблица

---



In [ ]:
results <- data.frame(
  Method = c("Теория", "Эксперимент", "LPT"),
  Wsyst = c(Wsyst, Wsyst2, Wsyst3)
)

print(knitr::kable(results, align = "c", caption = "Wsyst"))



Table: Wsyst

|   Method    |  Wsyst   |
|:-----------:|:--------:|
|   Теория    | 3.555560 |
| Эксперимент | 3.593829 |
|     LPT     | 3.368749 |
